In [41]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [42]:
dataset = 'AMK2021'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/pr1c00447_si_002.csv', header=2)
print('Raw data loaded.')
data

Loading raw data for AMK2021 ...
Raw data loaded.


,ID,Ctrl_p1,45I_p1,120R_p1,Ctrl_p3,45I_p3,30R_p3,120R_p3,Ctrl_p5,45I_p5,...,ANOVA Significant,ProteinID,Positions within proteins,Leading proteins,Protein,Protein Names,Gene Names,Sequence window,Unique identifier,Cluster
0,ACIN1|S|216|___1,0.462769,1.243250,1.370163,0.800144,0.840578,-3.346778,0.625082,-0.253426,-0.961845,...,+,Q9UKV3,216,Q9UKV3,Q9UKV3,Apoptotic chromatin condensation inducer in th...,ACIN1,HSPRKSSSISEEKGDSDDEKPRKGERRSSRV,UID1633,cluster1
1,C11orf84|S|251|___2,-3.844338,-3.012859,-2.116707,-4.202477,-2.457833,-3.374892,-1.091356,-2.915642,-1.994839,...,+,Q9BUA3,251,Q9BUA3,Q9BUA3,Uncharacterized protein C11orf84,C11orf84,MTKNLDPDPEPPSPDSPTETFAAPAEVRHFT,UID3358,cluster1
2,CANX|S|554|___2,-2.032246,-3.033451,-1.385906,-2.482878,-3.704783,-3.553141,-2.842339,-3.054946,-3.583059,...,+,P27824,554,P27824,P27824,Calnexin,CANX,DEEEEGEEKLEEKQKSDAEEDGGTVSQEEED,UID2411,cluster1
3,CBX3|S|93|___1,-3.432615,-2.958384,0.292416,-3.497353,-3.681576,-3.465493,-0.097929,-3.408276,-2.579869,...,+,Q13185,93,Q13185,Q13185,Chromobox protein homolog 3,CBX3,NSQKAGKEKDGTKRKSLSDSESDDSKSKKKR,UID759,cluster1
4,CBX3|S|93|___2,-1.680220,-0.529900,0.899700,-1.361748,-3.244538,-3.016910,0.886286,-2.453209,-1.658747,...,+,Q13185,93,Q13185,Q13185,Chromobox protein homolog 3,CBX3,NSQKAGKEKDGTKRKSLSDSESDDSKSKKKR,UID2758,cluster1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,TMPO|T|160;160|___2,1.096296,0.761822,-0.720869,0.528145,0.155390,-2.074714,-0.309784,1.543734,1.423948,...,+,P42167;P42166,160;160,P42167,P42167,"Lamina-associated polypeptide 2, isoforms beta...",TMPO,KLLKLREQGTESRSSTPLPTISSSAENTRQN,UID3858,cluster6
301,TNKS1BP1|S|1620|___2,3.489992,3.049110,2.794607,3.211349,3.208344,2.894361,2.881805,3.449987,2.985741,...,+,Q9C0C2,1620,Q9C0C2,Q9C0C2,182 kDa tankyrase-1-binding protein,TNKS1BP1,HLFQDSTEPRASRVPSSDEEVVEEPQSRRTR,UID3406,cluster6
302,TNKS1BP1|S|1621|___2,3.489992,3.049110,2.794607,3.211349,3.208344,2.894361,2.881805,3.449987,2.985741,...,+,Q9C0C2,1621,Q9C0C2,Q9C0C2,182 kDa tankyrase-1-binding protein,TNKS1BP1,LFQDSTEPRASRVPSSDEEVVEEPQSRRTRM,UID3407,cluster6
303,WDR44|S|3|___2,-2.151237,-2.329964,-3.421786,0.120436,-3.517701,-4.376318,-1.765232,0.232294,-3.327224,...,+,Q5JSH3,3,Q5JSH3,Q5JSH3,WD repeat-containing protein 44,WDR44,_____________MASESDTEEFYDAPEDVH,UID2913,cluster6


In [43]:
data.columns = data.columns.str.strip()

In [44]:
# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino acid'].str.contains(';', na=False)]
data = data[~data['Positions  within proteins'].str.contains(';', na=False)]
data = data[~data['Gene Names'].str.contains(';', na=False)]
data

,ID,Ctrl_p1,45I_p1,120R_p1,Ctrl_p3,45I_p3,30R_p3,120R_p3,Ctrl_p5,45I_p5,...,ANOVA Significant,ProteinID,Positions within proteins,Leading proteins,Protein,Protein Names,Gene Names,Sequence window,Unique identifier,Cluster
0,ACIN1|S|216|___1,0.462769,1.243250,1.370163,0.800144,0.840578,-3.346778,0.625082,-0.253426,-0.961845,...,+,Q9UKV3,216,Q9UKV3,Q9UKV3,Apoptotic chromatin condensation inducer in th...,ACIN1,HSPRKSSSISEEKGDSDDEKPRKGERRSSRV,UID1633,cluster1
1,C11orf84|S|251|___2,-3.844338,-3.012859,-2.116707,-4.202477,-2.457833,-3.374892,-1.091356,-2.915642,-1.994839,...,+,Q9BUA3,251,Q9BUA3,Q9BUA3,Uncharacterized protein C11orf84,C11orf84,MTKNLDPDPEPPSPDSPTETFAAPAEVRHFT,UID3358,cluster1
2,CANX|S|554|___2,-2.032246,-3.033451,-1.385906,-2.482878,-3.704783,-3.553141,-2.842339,-3.054946,-3.583059,...,+,P27824,554,P27824,P27824,Calnexin,CANX,DEEEEGEEKLEEKQKSDAEEDGGTVSQEEED,UID2411,cluster1
3,CBX3|S|93|___1,-3.432615,-2.958384,0.292416,-3.497353,-3.681576,-3.465493,-0.097929,-3.408276,-2.579869,...,+,Q13185,93,Q13185,Q13185,Chromobox protein homolog 3,CBX3,NSQKAGKEKDGTKRKSLSDSESDDSKSKKKR,UID759,cluster1
4,CBX3|S|93|___2,-1.680220,-0.529900,0.899700,-1.361748,-3.244538,-3.016910,0.886286,-2.453209,-1.658747,...,+,Q13185,93,Q13185,Q13185,Chromobox protein homolog 3,CBX3,NSQKAGKEKDGTKRKSLSDSESDDSKSKKKR,UID2758,cluster1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,STIM1|S|618|___1,-1.258076,-3.376553,-2.147388,-1.291973,-3.524648,-2.164458,-3.783055,-2.055235,-4.015877,...,+,Q13586,618,Q13586,Q13586,Stromal interaction molecule 1,STIM1,APPGGSPHLDSSRSHSPSSPDPDTPSPVGDS,UID801,cluster6
301,TNKS1BP1|S|1620|___2,3.489992,3.049110,2.794607,3.211349,3.208344,2.894361,2.881805,3.449987,2.985741,...,+,Q9C0C2,1620,Q9C0C2,Q9C0C2,182 kDa tankyrase-1-binding protein,TNKS1BP1,HLFQDSTEPRASRVPSSDEEVVEEPQSRRTR,UID3406,cluster6
302,TNKS1BP1|S|1621|___2,3.489992,3.049110,2.794607,3.211349,3.208344,2.894361,2.881805,3.449987,2.985741,...,+,Q9C0C2,1621,Q9C0C2,Q9C0C2,182 kDa tankyrase-1-binding protein,TNKS1BP1,LFQDSTEPRASRVPSSDEEVVEEPQSRRTRM,UID3407,cluster6
303,WDR44|S|3|___2,-2.151237,-2.329964,-3.421786,0.120436,-3.517701,-4.376318,-1.765232,0.232294,-3.327224,...,+,Q5JSH3,3,Q5JSH3,Q5JSH3,WD repeat-containing protein 44,WDR44,_____________MASESDTEEFYDAPEDVH,UID2913,cluster6


In [45]:
# filter data
data['Sequence window'] = data['Sequence window'].str.replace('_', '')
data

,ID,Ctrl_p1,45I_p1,120R_p1,Ctrl_p3,45I_p3,30R_p3,120R_p3,Ctrl_p5,45I_p5,...,ANOVA Significant,ProteinID,Positions within proteins,Leading proteins,Protein,Protein Names,Gene Names,Sequence window,Unique identifier,Cluster
0,ACIN1|S|216|___1,0.462769,1.243250,1.370163,0.800144,0.840578,-3.346778,0.625082,-0.253426,-0.961845,...,+,Q9UKV3,216,Q9UKV3,Q9UKV3,Apoptotic chromatin condensation inducer in th...,ACIN1,HSPRKSSSISEEKGDSDDEKPRKGERRSSRV,UID1633,cluster1
1,C11orf84|S|251|___2,-3.844338,-3.012859,-2.116707,-4.202477,-2.457833,-3.374892,-1.091356,-2.915642,-1.994839,...,+,Q9BUA3,251,Q9BUA3,Q9BUA3,Uncharacterized protein C11orf84,C11orf84,MTKNLDPDPEPPSPDSPTETFAAPAEVRHFT,UID3358,cluster1
2,CANX|S|554|___2,-2.032246,-3.033451,-1.385906,-2.482878,-3.704783,-3.553141,-2.842339,-3.054946,-3.583059,...,+,P27824,554,P27824,P27824,Calnexin,CANX,DEEEEGEEKLEEKQKSDAEEDGGTVSQEEED,UID2411,cluster1
3,CBX3|S|93|___1,-3.432615,-2.958384,0.292416,-3.497353,-3.681576,-3.465493,-0.097929,-3.408276,-2.579869,...,+,Q13185,93,Q13185,Q13185,Chromobox protein homolog 3,CBX3,NSQKAGKEKDGTKRKSLSDSESDDSKSKKKR,UID759,cluster1
4,CBX3|S|93|___2,-1.680220,-0.529900,0.899700,-1.361748,-3.244538,-3.016910,0.886286,-2.453209,-1.658747,...,+,Q13185,93,Q13185,Q13185,Chromobox protein homolog 3,CBX3,NSQKAGKEKDGTKRKSLSDSESDDSKSKKKR,UID2758,cluster1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,STIM1|S|618|___1,-1.258076,-3.376553,-2.147388,-1.291973,-3.524648,-2.164458,-3.783055,-2.055235,-4.015877,...,+,Q13586,618,Q13586,Q13586,Stromal interaction molecule 1,STIM1,APPGGSPHLDSSRSHSPSSPDPDTPSPVGDS,UID801,cluster6
301,TNKS1BP1|S|1620|___2,3.489992,3.049110,2.794607,3.211349,3.208344,2.894361,2.881805,3.449987,2.985741,...,+,Q9C0C2,1620,Q9C0C2,Q9C0C2,182 kDa tankyrase-1-binding protein,TNKS1BP1,HLFQDSTEPRASRVPSSDEEVVEEPQSRRTR,UID3406,cluster6
302,TNKS1BP1|S|1621|___2,3.489992,3.049110,2.794607,3.211349,3.208344,2.894361,2.881805,3.449987,2.985741,...,+,Q9C0C2,1621,Q9C0C2,Q9C0C2,182 kDa tankyrase-1-binding protein,TNKS1BP1,LFQDSTEPRASRVPSSDEEVVEEPQSRRTRM,UID3407,cluster6
303,WDR44|S|3|___2,-2.151237,-2.329964,-3.421786,0.120436,-3.517701,-4.376318,-1.765232,0.232294,-3.327224,...,+,Q5JSH3,3,Q5JSH3,Q5JSH3,WD repeat-containing protein 44,WDR44,MASESDTEEFYDAPEDVH,UID2913,cluster6


In [46]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 
    


In [47]:
data = match_seq_to_genename(data, 'Sequence window')

HSPRKSSSISEEKGDSDDEKPRKGERRSSRV
Match found for sequence: ID: sp|Q9UKV3|ACINU_HUMAN
Name: sp|Q9UKV3|ACINU_HUMAN
Description: sp|Q9UKV3|ACINU_HUMAN Apoptotic chromatin condensation inducer in the nucleus OS=Homo sapiens OX=9606 GN=ACIN1 PE=1 SV=2
Number of features: 0
Seq('MWRRKHPRTSGGTRGVLSGNRGVEYGSGRGHLGTFEGRWRKLPKMPEAVGTDPS...GRR')
Gene name match: <re.Match object; span=(102, 110), match='GN=ACIN1'>
Match found: HSPRKSSSISEEKGDSDDEKPRKGERRSSRV -> ACIN1
MTKNLDPDPEPPSPDSPTETFAAPAEVRHFT
Match found for sequence: ID: sp|Q9BUA3|SPNDC_HUMAN
Name: sp|Q9BUA3|SPNDC_HUMAN
Description: sp|Q9BUA3|SPNDC_HUMAN Spindlin interactor and repressor of chromatin-binding protein OS=Homo sapiens OX=9606 GN=SPINDOC PE=1 SV=3
Number of features: 0
Seq('MALKAEGAALDCFEVTLKCEEGEDEEEAMVVAVIPRPEPMLRVTQQEKTPPPRP...NGV')
Gene name match: <re.Match object; span=(109, 119), match='GN=SPINDOC'>
Match found: MTKNLDPDPEPPSPDSPTETFAAPAEVRHFT -> SPINDOC
DEEEEGEEKLEEKQKSDAEEDGGTVSQEEED
Match found for sequence: ID: sp|P2

In [48]:
data['Phosphosite'] = data['Amino acid'].astype(str) + '(' + data['Positions  within proteins'].astype(str) + ')'


In [49]:
keepcols = ['Phosphosite', 'GeneName'] + [
    col for col in data.columns if '_p1' in col or '_p3' in col or '_p5' in col]
data = data[keepcols]
data

,Phosphosite,GeneName,Ctrl_p1,45I_p1,120R_p1,Ctrl_p3,45I_p3,30R_p3,120R_p3,Ctrl_p5,45I_p5,30R_p5,120R_p5
0,S(216),ACIN1,0.462769,1.243250,1.370163,0.800144,0.840578,-3.346778,0.625082,-0.253426,-0.961845,-3.916733,-0.460504
1,S(251),SPINDOC,-3.844338,-3.012859,-2.116707,-4.202477,-2.457833,-3.374892,-1.091356,-2.915642,-1.994839,-3.441255,-1.378628
2,S(554),CANX,-2.032246,-3.033451,-1.385906,-2.482878,-3.704783,-3.553141,-2.842339,-3.054946,-3.583059,-3.409923,-1.336300
3,S(93),CBX3,-3.432615,-2.958384,0.292416,-3.497353,-3.681576,-3.465493,-0.097929,-3.408276,-2.579869,-2.868431,0.404577
4,S(93),CBX3,-1.680220,-0.529900,0.899700,-1.361748,-3.244538,-3.016910,0.886286,-2.453209,-1.658747,-1.291752,-0.141241
...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,S(618),STIM1,-1.258076,-3.376553,-2.147388,-1.291973,-3.524648,-2.164458,-3.783055,-2.055235,-4.015877,-2.240601,-3.709255
301,S(1620),TNKS1BP1,3.489992,3.049110,2.794607,3.211349,3.208344,2.894361,2.881805,3.449987,2.985741,2.446466,3.017368
302,S(1621),TNKS1BP1,3.489992,3.049110,2.794607,3.211349,3.208344,2.894361,2.881805,3.449987,2.985741,2.446466,3.017368
303,S(3),WDR44,-2.151237,-2.329964,-3.421786,0.120436,-3.517701,-4.376318,-1.765232,0.232294,-3.327224,-3.987392,-3.140080


In [50]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_2436/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [51]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains(';', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains('-', case=False, na=False)]

    # Add this line to remove decimals from phosphosite_ID (e.g., S123.0 -> S123)
    data['phosphosite_ID'] = data['phosphosite_ID'].apply(lambda x: re.sub(r'\((\d+)\.0+\)', r'(\1)', x))
    
    data_grouped = data.groupby(by='phosphosite_ID')
    
    if len(data) != len(data_grouped):
        numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
        non_numeric_cols = data.columns.difference(numeric_cols + ['phosphosite_ID']).tolist()
        data_numeric = data_grouped[numeric_cols].mean()
        data_categorical = data_grouped[non_numeric_cols].first().reset_index()
        
        # Merge numeric and non-numeric parts
        data = pd.merge(data_categorical, data_numeric, on='phosphosite_ID')
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')

    # Replace inf values with NaNs
    data = data.replace([np.inf, -np.inf], np.nan)
    
    # Ensure phosphosite_ID is first column
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)

    return data


In [52]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

Phosphosites with multiple measurements have been averaged
After cleaning phosphosite_ID column:


,phosphosite_ID,Ctrl_p1,45I_p1,120R_p1,Ctrl_p3,45I_p3,30R_p3,120R_p3,Ctrl_p5,45I_p5,30R_p5,120R_p5
0,AAK1_S(623),2.462610,1.698994,1.886770,2.318422,1.539989,2.190622,2.121202,2.263382,2.001007,1.878067,1.994272
1,AAK1_T(620),2.462610,1.698994,1.886770,2.318422,1.539989,2.190622,2.121202,2.263382,2.001007,1.878067,1.994272
2,ABCF1_S(109),-0.028277,-3.091283,-0.265938,-0.951149,-3.825162,-0.434930,-0.390485,-0.140622,-3.120581,-1.775134,0.024941
3,ABCF1_S(140),-2.525637,-3.726699,-3.137019,-0.897154,-2.866150,-2.648697,-4.004894,-0.718014,-3.962219,-4.031953,-3.325645
4,ABCF1_T(108),-0.028277,-3.091283,-0.265938,-0.973183,-3.265168,-0.434930,-0.495491,-1.737122,-3.716563,-1.907550,0.024941
...,...,...,...,...,...,...,...,...,...,...,...,...
268,WDR44_S(5),-2.151237,-2.796428,-3.171749,0.120436,-3.474931,-3.709595,-3.558636,0.232294,-2.672986,-2.793059,-2.785678
269,WIZ_S(1012),-2.054426,-3.340715,-3.381103,-2.648025,-4.024471,-2.065815,-2.437185,-2.688081,-3.340231,-2.298897,-3.364247
270,ZC3HAV1_S(275),-0.877483,-3.658138,-0.960880,-0.838757,-2.823757,-2.266857,-1.117985,-1.049505,-3.336305,-4.585115,-1.174196
271,ZC3HAV1_T(273),-0.877483,-3.458579,-0.960880,-0.838757,-3.053225,-2.963328,-1.117985,-1.049505,-2.082336,-3.554265,-1.174196


In [53]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/AMK2021.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

AMK2021 has been saved to CSV successfully!      phosphosite_ID   Ctrl_p1    45I_p1   120R_p1   Ctrl_p3    45I_p3  \
0       AAK1_S(623)  2.462610  1.698994  1.886770  2.318422  1.539989   
1       AAK1_T(620)  2.462610  1.698994  1.886770  2.318422  1.539989   
2      ABCF1_S(109) -0.028277 -3.091283 -0.265938 -0.951149 -3.825162   
3      ABCF1_S(140) -2.525637 -3.726699 -3.137019 -0.897154 -2.866150   
4      ABCF1_T(108) -0.028277 -3.091283 -0.265938 -0.973183 -3.265168   
..              ...       ...       ...       ...       ...       ...   
268      WDR44_S(5) -2.151237 -2.796428 -3.171749  0.120436 -3.474931   
269     WIZ_S(1012) -2.054426 -3.340715 -3.381103 -2.648025 -4.024471   
270  ZC3HAV1_S(275) -0.877483 -3.658138 -0.960880 -0.838757 -2.823757   
271  ZC3HAV1_T(273) -0.877483 -3.458579 -0.960880 -0.838757 -3.053225   
272      ZYX_S(344) -0.420448 -2.142162 -0.040731  0.041473 -0.989138   

       30R_p3   120R_p3   Ctrl_p5    45I_p5    30R_p5   120R_p5  
0    2.190622